<a href="https://colab.research.google.com/github/DriftKing-byte/QA-Chatbot/blob/main/Airline_Ticket_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.563838513999997
GPU (s):
0.12208935200004589
GPU speedup over CPU: 29x


In [ ]:
import pandas as pd
import io
import numpy as np

In [ ]:
from google.colab import files

uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
airline_data = pd.read_csv(io.BytesIO(uploaded['airline.csv']))

In [ ]:
airline_data.columns

In [ ]:
y = airline_data.price

In [ ]:
pp_d1 = airline_data[['airline','source_city','departure_time','stops','arrival_time','destination_city', 'duration', 'days_left']]

In [ ]:
pp_d1.head()

In [ ]:
##these columns can be one hot encoded. The other columns should be either removed or extracted and then one hot encoded. Add all these encoded stuff
## to final X.
pp_d1_mod = pd.get_dummies(pp_d1, columns = ['airline','source_city','departure_time','stops','arrival_time','destination_city'])

In [ ]:
#check to see if 1 and 0 filled in all the columns
pp_d1_mod.head()

In [ ]:
#Didn't happen so we have to use the following code to change the type of output data
pp_d1_mod2 = pp_d1_mod.astype(int)

In [ ]:
pp_d1_mod2.head()

In [ ]:
## we have remaining 'flight' and 'class'; string extraction and encoding will be done to flight. ordinal encoding will be done to class.

In [ ]:
#to extract first part of string in flight column
pp_d1_mod2['flight'] = airline_data['flight'].str.split('-').str[0]

In [ ]:
pp_d1_mod2.head()

In [ ]:
pp_d3_mod = pd.get_dummies(pp_d1_mod2, columns = ['flight'])

In [ ]:
pp_d3_mod.head()

In [ ]:
pp_d3_mod = pp_d3_mod.astype(int)

In [ ]:
pp_d3_mod.head()

In [ ]:
#class is all that's left now. We have to change to ordinal ranking. We can do this using scikitlearn.
airline_data['class'].unique()


In [ ]:
from sklearn.preprocessing import OrdinalEncoder

class_order = [['Economy','Business']]
encoder = OrdinalEncoder(categories=class_order)


In [ ]:
pp_d3_mod['class'] = airline_data['class']

In [ ]:
pp_d3_mod.head()

In [ ]:
pp_d3_mod['class'] = encoder.fit_transform(pp_d3_mod[['class']])

In [ ]:
pp_d3_mod.head()

In [ ]:
# adding columns of one dataframe to another
pp_d3_mod[['duration', 'days_left']] = airline_data[['duration', 'days_left']]

In [ ]:
pp_d3_mod.head()

In [ ]:
pp_d3_mod['price'] = airline_data['price']

In [ ]:
pp_d3_mod.head()

In [ ]:
pp_d3_mod.dtypes

In [ ]:
pp_d3_mod.columns

In [ ]:
y = pp_d3_mod.price

In [ ]:
features = ['airline_AirAsia', 'airline_Air_India', 'airline_GO_FIRST',
       'airline_Indigo', 'airline_SpiceJet', 'airline_Vistara',
       'source_city_Bangalore', 'source_city_Chennai', 'source_city_Delhi',
       'source_city_Hyderabad', 'source_city_Kolkata', 'source_city_Mumbai',
       'departure_time_Afternoon', 'departure_time_Early_Morning',
       'departure_time_Evening', 'departure_time_Late_Night',
       'departure_time_Morning', 'departure_time_Night', 'stops_one',
       'stops_two_or_more', 'stops_zero', 'arrival_time_Afternoon',
       'arrival_time_Early_Morning', 'arrival_time_Evening',
       'arrival_time_Late_Night', 'arrival_time_Morning', 'arrival_time_Night',
       'destination_city_Bangalore', 'destination_city_Chennai',
       'destination_city_Delhi', 'destination_city_Hyderabad',
       'destination_city_Kolkata', 'destination_city_Mumbai', 'flight_6E',
       'flight_AI', 'flight_G8', 'flight_I5', 'flight_SG', 'flight_UK',
       'class', 'duration', 'days_left']

In [ ]:
X = pp_d3_mod[features]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= 42)

Below is using random forest regressor. Skip to below correlation heatmap for keras.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

airline_tree = RandomForestRegressor(random_state=1)

In [ ]:
# Takes 2 minutes to run
airline_tree.fit(X_train,y_train)

In [ ]:
airline_pred = airline_tree.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error

val_mae = mean_absolute_error(y_test, airline_pred)

In [ ]:
print("Validation MAE for Random Forest Model: {}".format(val_mae))

~2000 dollars difference. Random Forest Regressor works best for tabular. Non-tabular would be keras and tensorflow. XGBoost gives similar results. CNNs work best for non-tabular as said earlier. Try SVR

In [ ]:
from sklearn.preprocessing import StandardScaler
import cupy as cp

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

x_train_cp = cp.asarray(X_train_scaled)
x_test_cp = cp.asarray(X_test_scaled)
y_train_cp = cp.asarray(y_train)

In [ ]:
import cuml
from cuml.svm import SVR as cuSVR

model = cuSVR(kernel='rbf', C=1.0, epsilon=0.1)
model.fit(x_train_cp, y_train_cp)

In [ ]:
y_pred_cp = model.predict(x_test_cp)

y_pred_np = cp.asnumpy(y_pred_cp)

In [ ]:
from sklearn.metrics import mean_absolute_error

val_mae = mean_absolute_error(y_test, y_pred_np)

print(val_mae)

The above is GPU based algorithm for svr. Accuracy is still low.

Below is XGBoost implementation.

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

model = xgb.XGBRegressor(
    n_estimators=2000,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print(mae)

In [ ]:
from sklearn.svm import SVR

svr = SVR()

In [ ]:
svr.fit(X_train, y_train)

In [ ]:
y_pred = svr.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_test, y_pred))

In [ ]:
# Accuracy is used for classification but not regression models. Regression models have MAE instead among others but not accuracy, precision, etc.

In [ ]:
bill## Now, it is time to get the parameters and hypertune!
airline_tree.get_params()

Apparently SGD regressor works better than RandomTreeRegressor with data over 100K samples. There is more preprocessing to do probably and differs. So, find out what it is. You can hypertune that as well.

In [ ]:
## A heatmap shows non-linearity. Tree regressor seems much better than sgd regressor so it wasn't explored. The map can't be done before
#bcz everythong should be float/int

import seaborn as sns
import matplotlib.pyplot as plt

    # Assuming 'X' is your feature DataFrame and 'y' is your target Series
pp_d3_mod = pd.concat([X, y], axis=1)
correlation_matrix = pp_d3_mod.corr()
plt.figure(figsize=(55, 55))
sns.heatmap(correlation_matrix, annot_kws={"fontsize": 14, "fontweight": 'bold'}, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

print(correlation_matrix['price']) #print the correlation of each feature with price.

### Conclusion: 1) High MAE (i.e. 1070) 2) Low correlation of features to label 3) Runtime ~2 min.

### For future: 1) Find highly correlated data if possible and/or time allows (for a good project) 2) Allows for low MAE 3) Runtime might not change but is not issue now. Focus on hypertuning instead.

Below is keras and tensorflow application which is not accurate because their applications work best for non-tabular.

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(X_train)

In [ ]:
X_train = scaler.transform(X_train)

In [ ]:
X_test = scaler.fit_transform(X_test)

In [ ]:
X_train.max()

In [ ]:
X_test.max()

In [ ]:
X_train.min()

In [ ]:
X_test.min()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
from keras.optimizers import Adam, RMSprop

model = Sequential()
model.add(Dense(42, activation='relu'))
model.add(Dense(21, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss='mse')

In [ ]:
model.fit(X_train, y_train, batch_size = 128, epochs = 100)